In [1]:
!pip install nlpaug
!pip install transformers
!pip install swifter

     |████████████████████████████████| 389kB 14.1MB/s 
     |████████████████████████████████| 1.3MB 12.9MB/s 
     |████████████████████████████████| 1.1MB 44.4MB/s 
     |████████████████████████████████| 890kB 46.5MB/s 
     |████████████████████████████████| 2.9MB 60.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=d77650caaeb6d2de22b91d4971a396fc7d3fd326278cb1249e502e9b8e33e83b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 634kB 14.8MB/s 
     |████████████████████████████████| 471kB 28.0MB/s 
     |████████████████████████████████| 542kB 39.8MB/s 
     |████████████████████████████████| 92kB 12.5MB/s 
     |████████████████████████████████| 17.2MB 363kB/s 
     |████████████████████████████████| 23.1MB 82.0MB/s 
     |████████████████████████████████| 1.3MB 44.4MB/s 
     |████████████████████████

In [2]:
import itertools
from typing import List, Dict, Union

import nlpaug.augmenter.word as naw
import pandas as pd
import swifter

In [3]:
ROOT_PATH = "/content/drive/My Drive/Magíster 2020"

In [4]:
puj_train = pd.read_json(f"{ROOT_PATH}/Ordenado/new_train_test_data/puj_train_70.json", orient="records")
puj_test = pd.read_json(f"{ROOT_PATH}/Ordenado/new_train_test_data/puj_test_30.json", orient="records")

In [5]:
aurora_train = pd.read_json(f"{ROOT_PATH}/Ordenado/new_train_test_data/aurora_train_70.json", orient="records")
aurora_test = pd.read_json(f"{ROOT_PATH}/Ordenado/new_train_test_data/aurora_test_30.json", orient="records")

In [6]:
def calculate_augmentation(max_count: int, current_count: int) -> int:
    return (max_count - current_count) // current_count

In [7]:
AURORA_MAX = aurora_train["first_sdg"].value_counts().max()

In [ ]:
def augment_abstract(row: pd.Series, current_value_counts: pd.Series) -> List[Dict[str, Union[str, int]]]:
    aug = naw.ContextualWordEmbsAug(model_path="distilbert-base-cased", action="substitute")
    sdg = row["first_sdg"]
    title = row["title"]
    current_count = current_value_counts[row["first_sdg"]]
    if current_count == AURORA_MAX:
        return []
    new_samples = calculate_augmentation(AURORA_MAX, current_count)
    augmented_samples = aug.augment(row["clean_abstract"], n=new_samples)
    augmented_samples = [{"title": title, "first_sdg": sdg, "clean_abstract": sample} for sample in augmented_samples]
    return augmented_samples

In [ ]:
puj_value_counts = puj_train["first_sdg"].value_counts()
puj_train["augmented"] = puj_train.apply(lambda row: augment_abstract(row, puj_value_counts), axis=1)

In [ ]:
augmented_flat = list(itertools.chain(*puj_train["augmented"].tolist()))
augmented = pd.DataFrame(augmented_flat)

In [ ]:
del puj_train["augmented"]
puj_train = pd.concat([puj_train, augmented], ignore_index=True)

In [ ]:
puj_train.to_json(f"{ROOT_PATH}/Ordenado/augmented_with_aurora/2020-09-12_puj_augmented.json", orient="records")

In [ ]:
 puj_train.head()

,title,clean_abstract,first_sdg
0,Sicobio Consolidation And Analysis System Of B...,Improving information systems and guaranteeing...,1
1,Effects Of Overfeeding And High-Fat Diet On Ca...,Obesity is a complex global health problem bec...,2
2,Advances In Medicinal Plants With Effects On A...,Background: Many mental health conditions incl...,3
3,Frailty And Sarcopenia In Bogotá: Results From...,Background: Latin American countries like Colo...,3
4,Wild And Domestic Animals Likely Involved In R...,"Between 2006 and 2008, three outbreaks of huma...",3


In [ ]:
puj_train["first_sdg"].value_counts()

7     1185
8     1185
5     1185
14    1185
16    1185
12    1185
10    1184
11    1184
6     1184
13    1183
17    1183
4     1182
15    1182
9     1178
1     1176
2     1170
3     1005
Name: first_sdg, dtype: int64